In [2]:
!pip install transformers datasets sacremoses torch torcheval
# !pip install negate torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 24.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.wh

In [33]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.1 MB/s eta 0:00:00


In [3]:
from numpy.random import RandomState
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForQuestionAnswering
# from negate import Negator
import torch

In [ ]:
# NOTE: do not need to run this anymore
negator = Negator(use_transformers=False)

def save_dataset(dataset, filename):
    dataset.to_csv(filename, index=False)

def negate_question(question):
    return negator.negate_sentence(question)

def augment_dataset(dataset, dataset_name):
    augmented_questions = []
    for data in dataset['train']:
        original_question = data['question']
        original_label = data['final_decision'] if dataset_name == 'pubmedQA' else data['answer']
        original_context = data['context']

        negated_question = negate_question(original_question)
        negated_label = 'no' if original_label == 'yes' else 'yes'

        augmented_questions.append({
            'question': original_question,
            'label': original_label,
            'context': original_context
        })

        augmented_questions.append({
            'question': negated_question,
            'label': negated_label,
            'context': original_context
        })

    return augmented_questions

def convert_row_to_sentence(datapoint):
    symptoms = [col.replace('_', ' ') for col in disease_symptoms_dataset.columns if datapoint[col] == 1]
    sentence = None
    if len(symptoms) > 1:
      sentence = ", ".join(symptoms[:-1]) + ", and " + symptoms[-1]
      sentence = f"The patient has {sentence}."
    elif symptoms:
      sentence = f"The patient has {symptoms[0]}."
    else:
      sentence = "The patient has no listed symptoms."
    return sentence

disease_symptoms_dataset['sentence'] = disease_symptoms_dataset.apply(convert_row_to_sentence, axis=1)
augmented_disease_symptoms_dataset = disease_symptoms_dataset[['sentence', 'prognosis']]
save_dataset(augmented_disease_symptoms_dataset, 'augmented_disease_symptoms.csv')

augmented_bioasq = augment_dataset(bioasq_dataset, 'bioasq')
augmented_bioasq_dataset = pd.DataFrame(augmented_bioasq)
save_dataset(augmented_bioasq_dataset, 'augmented_bioasq_dataset.csv')
print('Done augmenting dataset 1')

augmented_pubmedQA = augment_dataset(pubmedQA_dataset, 'pubmedQA')
augmented_pubmedQA_dataset = pd.DataFrame(augmented_pubmedQA)
save_dataset(augmented_pubmedQA_dataset, 'augmented_pubmedQA_dataset.csv')
print('Done augmenting dataset 2')


print(mednli_dataset, augmented_pubmedQA_dataset, augmented_bioasq_dataset, medQA_dataset, disease_symptoms_dataset)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
mednli_dataset = load_dataset('hippocrates/MedNLI_test')
pubmedQA_dataset = load_dataset("qiaojin/PubMedQA", "pqa_artificial")
bioasq_dataset = load_dataset("Coldog2333/bioasq")
medQA_dataset = load_dataset('bigbio/med_qa')
disease_symptoms_dataset = pd.read_csv('Disease_Training.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11232 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1422 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/1395 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

...

/root/.cache/huggingface/datasets/downloads/extracted/dc86b4af1d384a2680873fd3704ef3e09f2446bcfbf86dcc7655b8be63682696/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/dc86b4af1d384a2680873fd3704ef3e09f2446bcfbf86dcc7655b8be63682696/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/dc86b4af1d384a2680873fd3704ef3e09f2446bcfbf86dcc7655b8be63682696/data_clean/questions/US/dev.jsonl


In [7]:
# No need to run this again: Generate train and test datasets for bioasq

bioasq_df = pd.read_csv('augmented_bioasq_dataset.csv')

rng = RandomState()

bioasq_train = bioasq_df.sample(frac=0.7, random_state=rng)
bioasq_test = bioasq_df.loc[~bioasq_df.index.isin(bioasq_train.index)]

bioasq_train.to_csv('augmented_bioasq_dataset_train.csv', index=False)
bioasq_test.to_csv('augmented_bioasq_dataset_test.csv', index=False)

In [11]:
# No need to run this again: Generate train and test datasets for disease symptoms
disease_df = pd.read_csv('augmented_disease_symptoms.csv')

disease_train = disease_df.sample(frac=0.7, random_state=rng)
disease_test = disease_df.loc[~disease_df.index.isin(disease_train.index)]

disease_train.to_csv('augmented_disease_symptoms_train.csv', index=False)
disease_test.to_csv('augmented_disease_symptoms_test.csv', index=False)

In [ ]:
# No need to run this again: Generate train and test datasets for pubmedQA symptoms
pubmedQA_df = pd.read_csv('augmented_pubmedQA_dataset.csv')

pubmedQA_train = pubmedQA_df.sample(frac=0.7, random_state=rng)
pubmedQA_test = pubmedQA_df.loc[~pubmedQA_df.index.isin(pubmedQA_train.index)]

pubmedQA_train.to_csv('augmented_pubmedQA_dataset_train.csv', index=False)
pubmedQA_test.to_csv('augmented_pubmedQA_dataset_test.csv', index=False)

In [4]:
# Load train/test datasets from CSV

disease_train = pd.read_csv('augmented_disease_symptoms_train.csv')
disease_test = pd.read_csv('augmented_disease_symptoms_test.csv')

# bioasq_train = pd.read_csv('augmented_bioasq_dataset_train.csv')
# bioasq_test = pd.read_csv('augmented_bioasq_dataset_test.csv')

# pubmedQA_train = pd.read_csv('augmented_pubmedQA_dataset_train.csv')
# pubmedQA_test = pd.read_csv(' .csv')

# mednli_dataset_train = mednli_dataset['train']
# mednli_dataset_test = mednli_dataset['test']
# mednli_dataset_valid = mednli_dataset['valid']

# medQA_dataset_train = medQA_dataset['train']
# medQA_dataset_test = medQA_dataset['test']
# medQA_dataset_valid = medQA_dataset['validation']



In [9]:
print(disease_train['prognosis'].value_counts())
print(len(disease_train['prognosis'].value_counts()))

prognosis
hepatitis A                                94
Gastroenteritis                            93
Osteoarthristis                            92
Malaria                                    91
AIDS                                       91
Cervical spondylosis                       91
Varicose veins                             91
Hypertension                               90
Bronchial Asthma                           89
Drug Reaction                              88
Hypoglycemia                               87
Migraine                                   87
Common Cold                                87
Fungal infection                           87
Typhoid                                    86
Hepatitis B                                86
Allergy                                    86
Tuberculosis                               86
Heart attack                               86
Jaundice                                   85
Hepatitis C                                84
Urinary tract infection 

In [5]:
disease_biogpt_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
disease_biogpt_model = AutoModelForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=41, problem_type="multi_label_classification")

# disease_clinical_bert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# disease_clinical_bert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=41, problem_type="multi_label_classification")

# disease_bio_bert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
# disease_bio_bert_model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=41, problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = disease_train["sentence"][0]
encoded_input = disease_biogpt_tokenizer(text, return_tensors='pt')
output = disease_biogpt_model(**encoded_input)
print(output.logits)

tensor([[-0.5454,  0.6105,  0.5758,  0.8862,  0.5397, -0.7203, -2.4600, -0.6883,
         -0.2283,  0.4929,  0.0820, -0.8737, -0.2030, -0.7855,  0.1845, -1.5819,
         -1.4262, -0.5378,  0.2140, -0.0361,  0.0820, -0.7205,  0.1252, -0.4284,
          0.1872,  0.9838,  0.2031,  0.0392,  1.4296,  0.9432,  0.8852, -0.1635,
          0.1450, -0.7416,  0.9745, -0.0772, -0.0433, -0.5910,  0.9711, -0.1572,
         -0.7778]], grad_fn=<IndexBackward0>)


In [ ]:
text = disease_train["sentence"][0]
encoded_input = disease_clinical_bert_tokenizer(text, return_tensors='pt')
output = disease_clinical_bert_model(**encoded_input)
output.logits

tensor([[-0.2015,  0.2054,  0.2610,  0.1886, -1.0640, -0.1880,  0.3223,  0.1151,
          0.3266,  0.3804, -0.4181,  0.2671, -0.1243, -0.1631,  0.3998, -0.1877,
          0.3874, -0.4239,  0.3012, -0.0229,  0.3695,  0.1599,  0.2763, -0.5633,
          0.4563, -0.6643, -0.4651,  0.0662, -0.3678,  0.5397,  0.0579, -0.2181,
          0.4004, -0.0651,  0.6099,  0.5456, -0.1637,  0.4532,  0.5527, -0.7698,
         -0.1029]], grad_fn=<AddmmBackward0>)

In [ ]:
text = disease_train["sentence"][0]
encoded_input = disease_bio_bert_tokenizer(text, return_tensors='pt')
output = disease_bio_bert_model(**encoded_input)
output.logits

tensor([[-0.3033,  0.4545, -0.4315,  0.4206,  0.0585, -0.1273, -0.2455,  0.4104,
          0.0544, -0.1387,  0.3054,  0.2013,  0.3467, -0.1292,  0.1411, -0.3591,
         -0.3649, -0.0965, -0.0437, -0.4583,  0.1716,  0.1861, -0.3845, -0.4893,
         -0.0925, -0.5033,  0.1053,  0.7184,  0.0229, -0.1227,  0.7855,  0.8356,
         -0.0508,  0.0226, -0.1440,  0.1289, -0.2273, -0.3512, -0.2630, -0.3932,
         -0.0858]], grad_fn=<AddmmBackward0>)

In [6]:
BATCH_SIZE = 64
LR = 0.0001
OPTIMIZER = "adam"
EPOCHS = 10

In [8]:
# Disease Symptoms Training Loop
from torch.nn.functional import cross_entropy, one_hot
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import time
from torcheval.metrics import MulticlassAccuracy

device = "cuda" if torch.cuda.is_available() else "cpu"

def loss_fn(y_hat, y):
    return cross_entropy(y, y_hat)

def change_y_train(y_train):
    mapping = {}
    prevMapping = 0
    for label in y_train:
        if label not in mapping:
            mapping[label] = prevMapping
            prevMapping += 1

    for i in range(len(y_train)):
        y_train[i] = mapping[y_train[i]]

    labels = y_train.values.astype(np.int64)
    res = one_hot(torch.from_numpy(labels), num_classes=41)
    return res, mapping

def change_y_test(y_test, mapping):
    for i in range(len(y_test)):
        y_test[i] = mapping[y_test[i]]

    labels = y_test.values.astype(np.int64)
    res = one_hot(torch.from_numpy(labels), num_classes=41)
    return res

def train(model):
    model = model.to(device)

    X_train = disease_train["sentence"]
    y_train, mapping = change_y_train(disease_train["prognosis"])

    print(y_train)

    X_test = disease_test["sentence"]

    y_test = change_y_test(disease_test["prognosis"], mapping)

    print(y_test)

    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    X_train = X_train.values.tolist()
    X_valid = X_valid.values.tolist()
    y_train = y_train.to(device)
    y_valid = y_valid.to(device)

    batch_idx = torch.arange(0, len(X_train), BATCH_SIZE)
    print(batch_idx)

    optimizer = Adam(model.parameters(), lr=LR)

    tr_losses = []
    val_losses = []
    val_acc = []

    for epoch in range(EPOCHS):
        model.train()
        training_loss = 0
        for i in batch_idx:
            print(i)
            j = i.item()
            X = X_train[j: j + BATCH_SIZE]
            y = y_train[j: j + BATCH_SIZE]

            encoded_input = disease_biogpt_tokenizer(X, return_tensors='pt', padding=True, truncation=True).to(device)
            output = disease_biogpt_model(**encoded_input).logits

            L = loss_fn(output, y)
            training_loss += len(X) * L

            optimizer.zero_grad()

            L.backward()

            optimizer.step()

        model.eval()

        with torch.no_grad():
            encoded_input = disease_biogpt_tokenizer(X_valid, return_tensors='pt')
            valid_output = disease_biogpt_model(**encoded_input).logits

            valid_loss = loss_fn(valid_output, y_valid)
            val_losses.append(valid_loss)

            metric = MulticlassAccuracy()
            metric.update(valid_output, y_valid)
            acc = metric.compute().item()

            tr_losses.append(training_loss / len(X_train))
            val_acc.append(acc)

            print('\tEPOCH: ', epoch, ', Training Loss: ', training_loss / len(X_train), ', Validation Loss: ', valid_loss, ', Validation Acc: ', acc)

    timestamp = time.strftime('%b-%d-%Y_%H%M', time.localtime())
    torch.save(model.state_dict(), type(model).__name__ + "_" + timestamp + ".pth")

    return tr_losses, val_losses, val_acc




# print(disease_train[:6])
print(type(disease_biogpt_model).__name__)
print(train(disease_biogpt_model))

BioGptForSequenceClassification
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]])
tensor([   0,   64,  128,  192,  256,  320,  384,  448,  512,  576,  640,  704,
         768,  832,  896,  960, 1024, 1088, 1152, 1216, 1280, 1344, 1408, 1472,
        1536, 1600, 1664, 1728, 1792, 1856, 1920, 1984, 2048, 2112, 2176, 2240,
        2304, 2368, 2432, 2496, 2560, 2624, 2688, 2752])
tensor(0)


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 3.06 MiB is free. Process 13231 has 14.74 GiB memory in use. Of the allocated memory 14.34 GiB is allocated by PyTorch, and 286.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
bioasq_biogpt_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
bioasq_biogpt_model = AutoModelForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=2, problem_type="multi_label_classification")

bioasq_clinical_bert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
bioasq_clinical_bert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=2, problem_type="multi_label_classification")

bioasq_bio_bert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
bioasq_bio_bert_model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=2, problem_type="multi_label_classification")

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = bioasq_train["context"][0] + bioasq_train["question"][0]

encoded_input = bioasq_biogpt_tokenizer(text, return_tensors='pt')
output = bioasq_biogpt_model(**encoded_input)
print(output.logits)

encoded_input = bioasq_clinical_bert_tokenizer(text, return_tensors='pt')
output = bioasq_clinical_bert_model(**encoded_input)
print(output.logits)

encoded_input = bioasq_bio_bert_tokenizer(text, return_tensors='pt')
output = bioasq_bio_bert_model(**encoded_input)
output.logits

tensor([[ 1.2313, -1.4946]], grad_fn=<IndexBackward0>)
tensor([[-0.2418,  0.2327]], grad_fn=<AddmmBackward0>)


tensor([[-0.2132,  0.1646]], grad_fn=<AddmmBackward0>)

In [ ]:
pubmedQA_biogpt_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
pubmedQA_biogpt_model = AutoModelForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=2, problem_type="multi_label_classification")

pubmedQA_clinical_bert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
pubmedQA_clinical_bert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=2, problem_type="multi_label_classification")

pubmedQA_bio_bert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
pubmedQA_bio_bert_model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=2, problem_type="multi_label_classification")

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = pubmedQA_train["context"][4] + pubmedQA_train["question"][4]

encoded_input = pubmedQA_biogpt_tokenizer(text, return_tensors='pt')
output = pubmedQA_biogpt_model(**encoded_input)
print(output.logits)

encoded_input = pubmedQA_clinical_bert_tokenizer(text, return_tensors='pt')
output = pubmedQA_clinical_bert_model(**encoded_input)
print(output.logits)

encoded_input = pubmedQA_bio_bert_tokenizer(text, return_tensors='pt')
output = pubmedQA_bio_bert_model(**encoded_input)
output.logits

tensor([[-0.7861, -0.5916]], grad_fn=<IndexBackward0>)
torch.Size([1, 402])
torch.Size([1, 402])
tensor([[ 0.4580, -0.0365]], grad_fn=<AddmmBackward0>)


tensor([[ 0.2339, -0.4015]], grad_fn=<AddmmBackward0>)

In [ ]:
mednli_biogpt_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
mednli_biogpt_model = AutoModelForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=3, problem_type="multi_label_classification")

mednli_clinical_bert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
mednli_clinical_bert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=3, problem_type="multi_label_classification")

mednli_bio_bert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
mednli_bio_bert_model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=3, problem_type="multi_label_classification")


Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = mednli_dataset_train["query"][4]

encoded_input = mednli_biogpt_tokenizer(text, return_tensors='pt')
output = mednli_biogpt_model(**encoded_input)
print(output.logits)

encoded_input = mednli_clinical_bert_tokenizer(text, return_tensors='pt')
output = mednli_clinical_bert_model(**encoded_input)
print(output.logits)

encoded_input = mednli_bio_bert_tokenizer(text, return_tensors='pt')
output = mednli_bio_bert_model(**encoded_input)
output.logits

tensor([[-0.3662,  0.3433, -1.0313]], grad_fn=<IndexBackward0>)
tensor([[ 0.3764, -0.0981,  0.2205]], grad_fn=<AddmmBackward0>)


tensor([[0.2935, 0.2968, 0.0638]], grad_fn=<AddmmBackward0>)

In [ ]:
medQA_biogpt_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
medQA_biogpt_model = AutoModelForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=5, problem_type="multi_label_classification")

medQA_clinical_bert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
medQA_clinical_bert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=5, problem_type="multi_label_classification")

medQA_bio_bert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
medQA_bio_bert_model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=5, problem_type="multi_label_classification")


Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = medQA_dataset_train["question"][4] + ''.join(list(medQA_dataset_train["options"][4][0].values()))

encoded_input = medQA_biogpt_tokenizer(text, return_tensors='pt')
output = medQA_biogpt_model(**encoded_input)
print(output.logits)

encoded_input = medQA_clinical_bert_tokenizer(text, return_tensors='pt')
output = medQA_clinical_bert_model(**encoded_input)
print(output.logits)

encoded_input = medQA_bio_bert_tokenizer(text, return_tensors='pt')
output = medQA_bio_bert_model(**encoded_input)
output.logits

tensor([[-1.3896,  0.2885, -0.8583,  0.5450,  0.9272]],
       grad_fn=<IndexBackward0>)
tensor([[-0.3869,  0.2765, -0.1143,  0.0390, -0.5103]],
       grad_fn=<AddmmBackward0>)


tensor([[ 0.3877,  0.6230, -0.4189, -0.0960, -0.1126]],
       grad_fn=<AddmmBackward0>)